In [1]:
#############
###paths#####
#############
import sys
import os

pwd = os.getcwd()
modellingpath = pwd.rpartition("modelling")[0] + pwd.rpartition("modelling")[1] 
sys.path.append(modellingpath + '/lib')
#############
import pickle as pkl
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from equations.class_subcircuit_eq import *
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [9]:

# Specify name of circuit and variant investigated
circuit_n='circuit14'
variant='1nd'
# Specifiy number of parameter sets in parameterset file to be loaded
n_param_sets = 10

print(f'Circuit:{circuit_n}, Variant:{variant}')

# df_full= pkl.load( open(modellingpath + "/3954/paper/input/lhs_parameterfiles/df_%s_variant%s_%rparametersets.pkl"%(circuit_n,variant,n_param_sets), "rb"))
df_full= pkl.load( open(modellingpath + "/3954/paper/input/balanced_parameterfiles/df_%s_variant%s_%rparametersets_balanced.pkl"%(circuit_n,variant,n_param_sets), "rb"))
df_full = df_full.drop(['balance'], axis=1)
df_full

Circuit:circuit14, Variant:1nd


,Dr,VA,VB,VC,VD,VE,VF,Kda,Kab,Keb,...,Kce,muLVA,muASV,nbd,nab,nda,nfe,nee,neb,nce
0,3.301474,474.138604,100.007619,19.375832,122.339359,469.329591,45.044615,2.491955,90.556145,6.299995,...,1.311751,3.458077,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
1,16.308159,854.590091,112.239410,25.711948,34.413164,18.998190,855.557185,7.238064,28.244814,1.734142,...,2.214423,4.464250,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
2,9.547771,43.054779,52.995127,233.816237,562.037502,277.238209,90.260879,201.985902,6.685313,40.338746,...,4.765898,3.789006,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
3,0.070026,169.504705,128.159017,189.279316,695.321372,308.940218,301.709580,6.497529,3.449321,10.885814,...,1.657792,3.757268,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
4,0.469120,119.774787,54.978802,541.287549,598.577284,469.861937,30.528432,5.834824,3.616880,2.364029,...,82.835717,4.302146,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
5,0.981527,656.948185,217.703270,309.299785,84.425802,988.743800,201.539746,12.907976,106.600858,621.601444,...,19.742098,3.543059,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
6,0.042112,44.349196,46.035933,357.946933,16.971478,803.841095,944.272100,4.619097,21.314509,594.548632,...,19.002336,4.212674,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
7,92.729579,63.739165,79.664311,564.813006,194.944158,22.074748,168.939281,1.688920,60.325814,13.269466,...,90.997370,3.731697,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
8,1.105871,58.517565,67.956774,36.440014,463.923977,126.479182,71.295751,26.983105,6.641606,13.235071,...,7.941957,4.092195,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0
9,35.692866,115.414263,493.475023,829.214128,278.516231,355.398539,131.958768,7.871625,3.658475,354.333526,...,30.344885,4.239906,1.0,2.0,1.0,1.0,4.0,4.0,4.0,1.0


In [3]:
def checkBalance(par_dict):
    balanceDict = {}
    for Km in Km_list:
        # print(Km)
        Vx =par_dict[KtoV[Km]]
        Kxy = par_dict[Km]
        if Kxy >= 1 and Kxy <= Vx:
            # print(Km, 'Balanced')
            balanceDict[Km] = 'Balanced'
        elif Kxy > 0.1 and Kxy < Vx*10:
            # print(Km, 'Semi balanced')
            balanceDict[Km] ='Semi balanced'
            # balanceDict[Km] = 'Balanced'

        elif Kxy <= 0.1 or Kxy >= Vx*10:
            # print(Km, 'Not balanced')
            balanceDict[Km] ='Not balanced'
            # balanceDict[Km] = 'Balanced'

        else:
            print('ERROR!!!!!!!!!')
        
    # print('--------------')

    if 'Not balanced' in balanceDict.values():
        return 'Not balanced'
    elif 'Semi balanced'  in balanceDict.values():
        return 'Semi balanced'
    elif all(x == 'Balanced' for x in balanceDict.values()):
        return 'Balanced'
    
#removed kee from balancing
Km_list = ['Kda', 'Kab', 'Keb', 'Kbd', 'Kfe',  'Kce' ]
KtoV = {'Kda': 'VD', 'Kab': 'VA', 'Keb': 'VE', 'Kbd': 'VB', 'Kfe': 'VF', 'Kce': 'VC' }
balanceList = []    
for parID in tqdm(df_full.index):
    par_dict = df_full.loc[parID].to_dict()
    balanceList.append(checkBalance(par_dict))
# print(balanceList)
df_full['balance'] = balanceList



print(df_full['balance'].value_counts())
# df_full[df_full['balance'] == 'Balanced']
# pkl.dump(df_full, open(modellingpath + "/3954/paper/input/balanced_parameterfiles/df_%s_variant%s_%rparametersets_balanced.pkl"%(circuit_n,variant,n_param_sets), "wb"))


100%|██████████| 10/10 [00:00<00:00, 3307.81it/s]

Semi balanced    10
Name: balance, dtype: int64


In [4]:
df_full['balance'].value_counts()

Semi balanced    10
Name: balance, dtype: int64

In [5]:

Km_list = ['Kda', 'Kab', 'Keb', 'Kbd', 'Kfe', 'Kee', 'Kce' ]
KtoV = {'Kda': 'VD', 'Kab': 'VA', 'Keb': 'VE', 'Kbd': 'VB', 'Kfe': 'VF', 'Kee': 'VE', 'Kce': 'VC' }
for Km in Km_list:
   Vx =par_dict[KtoV[Km]]
   par_dict[Km]=Vx
print(checkBalance(par_dict))


Balanced
